In [2]:
import os
import numpy as np
import cv2
import tensorflow as tf
print(tf.__version__)

/Users/sreeharirammohan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.0


In [3]:
#X = np.load('/Volumes/Elements/CES-Model_and_Data/X.npy')
#y = np.load('/Volumes/Elements/CES-Model_and_Data/y.npy')

In [ ]:
#Read in images from folder
DATA_PATH = "/Volumes/Sreehari 4TB backup/Lesions/"

classes = os.listdir(DATA_PATH)
del classes[0] #first entry is .DS Store not a directory

print(classes)

X = np.array([])
y = np.array([])

MAX_PHOTOS_PER_CLASS = 100

for class_name in classes:
    
    print("Working on class", class_name)

    class_folder = os.path.join(DATA_PATH,class_name)

    sub_files = os.listdir(class_folder)
    
    number_of_files = len(sub_files)
    i = 0
    
    for file in sub_files:
        if '.jpg' in file:
            image_filepath = os.path.join(class_folder, file)
            image_as_np_array = cv2.imread(image_filepath,flags=cv2.IMREAD_COLOR)
            
            #(400, 400, 3)
            #decrease the size of the image by 1/4 to 
            image_as_np_array = cv2.resize(image_as_np_array, (400,400), interpolation = cv2.INTER_LINEAR)
            
            #cv2.imshow('ImageWindow', image_as_np_array)
            #cv2.waitKey()
            
            X = np.append(X, image_as_np_array)
            y = np.append(y, class_name)

            print(str(i) + " - " + str(round((i/MAX_PHOTOS_PER_CLASS)*100, 2)) + "%", end='\r', flush=True)

            i += 1
            
            if (i > MAX_PHOTOS_PER_CLASS):
                break
                
        


['Actinic_Keratosis', 'Basel_Cell_Carcinoma', 'Dermatofibroma', 'Melanoma', 'Nevus', 'Seaborrheic Keratosis']
Working on class Actinic_Keratosis
Working on class Basel_Cell_Carcinoma
Working on class Dermatofibroma
Working on class Melanoma


In [ ]:
X = X.reshape(-1,400,400,3)
X.shape

In [ ]:
y.shape

In [ ]:
#Normalize image array values between 0-1
X /= 255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)

In [ ]:
print(y)

In [ ]:
np.save('/Volumes/Elements/CES-Model_and_Data/X.npy', X)
np.save('/Volumes/Elements/CES-Model_and_Data/y.npy', y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
print(y_test)

In [ ]:
import keras 
from keras.models import Sequential
from keras.layers import *

num_classes = 6
input_shape=(400,400,3)
epochs = 10
batch_size = 32

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          verbose=1)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [39]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    tf.keras.models.save_model(
        model,
        '/Volumes/Elements/Lesions/65%_model_10-9.h5',
        overwrite=True,
        include_optimizer=True
    )

In [40]:
converter = tf.contrib.lite.TocoConverter.from_keras_model_file('/Volumes/Elements/Lesions/65%_model_10-9.h5')
converter.post_training_quantize = True
converter.quantize_weights = True
tflite_model = converter.convert()
open("/Volumes/Elements/Lesions/65%_epoch_model_10-9_quantized_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


10624816

In [41]:
print("done")

done
